# ECE/CS 434 | MP2: DUET
<br />
<nav>
    <span class="alert alert-block alert-warning">Due on Wednesday Feb 26 11:59PM on Gradescope</span>
   
</nav><br> 


## Objective
In this MP, you will:
- Implement DUET algorithm to separate a mixture of N voice signals from received from two microphones

---
## Problem Overview
Consider a problem of separating N sources ($S_1$, $S_2$, ... $S_N$) from recordings on 2 microphones ($R_1$ and $R_2$).
According to DUET algorithm, you will need to perform the following steps:

- Calculate the short-time Fourier transform of two received signals to get the time-frequency spectrograms
- Calculate the ratio of the two time-frequency spectrograms to get relative delay and attenuation
- Cluster the time-frequency bins in the 2D space spanned by relative delay and attenuation
- Recover the original N signals based on the clustering results

You can refer to the original DUET paper in ICASSP 2000: "Blind separation of disjoint orthogonal signals: demixing N sources from 2 mixtures" and this tutorial in Blind speech separation, 2007 - Springer: "The DUET blind source separation algorithm"

For the sake of easier clustering, the exact number of sources N will be provided to you.

You can assume there is no time-frequency bin collision for any two sources.

---
## Imports & Setup
To run the grading script of this MP, you will need to install the Python [SpeechRecognition](https://pypi.org/project/SpeechRecognition/) package. The SpeechRecognition package also requires the dependency [pocketsphinx](https://pypi.org/project/pocketsphinx/). You may directly use pip install to install both packages.
The following `code` cell, when run, imports the libraries you might need for this MP. Feel free to delete or import other commonly used libraries. Double check with the TA if you are unsure if a library is supported.

In [1]:
import numpy as np
import scipy
import scipy.io.wavfile
import scipy.signal
import speech_recognition as sr
import sklearn
import sklearn.cluster as clustersk
import sklearn.preprocessing

if __name__ == '__main__':
    import matplotlib.pyplot as plt
    plt.style.use("seaborn") # This sets the matplotlib color scheme to something more soothing
    from IPython import get_ipython
    get_ipython().run_line_magic('matplotlib', 'inline')

# This function is used to format test results. You don't need to touch it.
def display_table(data):
    from IPython.display import HTML, display
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4><td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

### Sanity-check

Running the following code block verifies that the correct module versions are indeed being used. 

Try restarting the Python kernel (or Jupyter) if there is a mismatch even after intalling the correct version. This might happen because Python's `import` statement does not reload already-loaded modules even if they are updated.

In [2]:
if __name__ == '__main__':
    from IPython.display import display, HTML, Audio

    def printc(text, color):
        display(HTML("<text style='color:{};weight:700;'>{}</text>".format(color, text)))

    _requirements = [r.split("==") for r in open(
        "packages.txt", "r").read().split("\n")]

    import sys
    for (module, expected_version) in _requirements:
        try:
            if sys.modules[module].__version__ != expected_version:
                printc("[✕] {} version should to be {}, but {} is installed.".format(
                    module, expected_version, sys.modules[module].__version__), "#f44336")
            else:
                printc("[✓] {} version {} is correct.".format(
                    module, expected_version), "#4caf50")
        except:
            printc("[–] {} is not imported, skipping version check.".format(
                module), "#03a9f4")

---
## Your Implementation
Implement your localization algorithm in the function `duet_source_separation(mic_data_folder, NUM_SOURCES)`. Do **NOT** change its function signature. You are, however, free to define and use helper functions. 

Your implementation for `duet_source_separation` function should **NOT** output any plots or data. It should only return the user's calculated location.

In [33]:
# np.set_printoptions(threshold=sys.maxsize)

def duet_source_separation(mic_data_folder, NUM_SOURCES):
    """DUET source separation algorithm. Write your code here.

    Args:
        mic_data_folder: name of folder (without a trailing slash) containing 
                         two mic datafiles `0.wav` and `1.wav`.

    Returns:
        NUM_SOURCES * recording_length numpy array, where NUM_SOURCES is the number of sources,
        and recording_length is the original length of the recording (in number of samples)

    """
    
    source0_sample_rate, source0_data = scipy.io.wavfile.read(mic_data_folder + "/0.wav")
    source1_sample_rate, source1_data = scipy.io.wavfile.read(mic_data_folder + "/1.wav")
    

    if (NUM_SOURCES == 1):
        return np.array([ source0_data])
    
    
    ## Take the STFT on both mics 
    stft_0_frequency,stft_0_time, stft_0 = scipy.signal.stft(source0_data, fs = source0_sample_rate, nperseg = 1024,noverlap=512)
    stft_1_frequency, stft_1_time, stft_1 = scipy.signal.stft(source1_data, fs = source1_sample_rate,nperseg = 1024, noverlap = 512)
    stft_0_frequency[0] = 1
    ## Take the STFT on both mics
    
    print("Time length: " ,len(stft_0_time))
    print ("Frequency length : " , len(stft_0_frequency))
    print("stft_0 shape: ", stft_0.shape)
  
    ## solely for debugging
    stft_0_magnitude = np.abs(stft_0)
    stft_0_phase = np.angle(stft_0)
    stft_1_magnitude = np.abs(stft_1)
    stft_1_phase = np.angle(stft_1)
    
    stft_ratio = (stft_0 + 1e-16) / (stft_1 + 1e-16)
    phase_differences = np.imag(np.log(stft_ratio))
    magnitude_ratios = np.abs(stft_ratio)

    ## Convert to time_differences 
   
    time_differences = np.zeros_like(phase_differences)
    for i in range(time_differences.shape[0]):
        for j in range(time_differences.shape[1]):
            time_differences[i,j] = phase_differences[i,j] / (-stft_0_frequency[i] )  #(2 * np.pi * stft_0_frequency[i])
          
                
                
    # for i in range(time_differences.shape[0]):
    #     for j in range(time_differences.shape[1]):
    #         if time_differences[i,j] > 0.001 or time_differences[i,j] < -0.001:
    #             time_differences[i,j] = 0
    # for i in range(magnitude_ratios.shape[0]):
    #     for j in range(magnitude_ratios.shape[1]):
    #         if magnitude_ratios[i,j] > 1.5 or magnitude_ratios[i,j] < 0.5:
    #             magnitude_ratios[i,j] = 0
    

    
    
    flattened_time_differences = time_differences.reshape( -1,1)
    flattened_phases = phase_differences.reshape(-1,1)
    
    #### CLustering #####
    
    # cluster_indices = bomber.fit_predict(flattened_time_differences).reshape(time_differences.shape)
    # scaler = sklearn.preprocessing.StandardScaler()
    # scaler.fit(flattened_time_differences)
    # flattened_time_differences = scaler.transform(flattened_time_differences)
    bomber  = clustersk.KMeans(n_clusters = NUM_SOURCES)
    bomb_labels = bomber.fit_predict(flattened_time_differences)
    print("cluster labels: ", bomb_labels)
    print("Cluster Centers: ",bomber.cluster_centers_)
    # bomber.fit(flattened_time_differences)
    ## Create Masks
    
    one_hots= np.zeros( (NUM_SOURCES, len(flattened_time_differences) ))
    for bomb_num in range(NUM_SOURCES):
        one_hots[bomb_num, bomb_labels == bomb_num] = 1
    one_hots =  one_hots.reshape( (NUM_SOURCES, len(stft_0_frequency), len(stft_0_time)))
    one_hot_transformed_stfts = np.zeros_like(one_hots, dtype=complex)
   
   ## Then multiply masks by stft
   
    for bomb_num in range(NUM_SOURCES):
        one_hot_burn = one_hots[bomb_num]
        one_hot_transformed_stfts[bomb_num] = stft_0 * one_hot_burn 
    _, inverted_stft = scipy.signal.istft(one_hot_transformed_stfts[0],fs=source0_sample_rate, nperseg = 1024,noverlap=512)
    shape_of_istft = inverted_stft.shape[0]
    
    ## Apply istft to separate sources (not working rn)
    
    output = np.zeros((NUM_SOURCES, shape_of_istft))
    for bomb_num in range(NUM_SOURCES):
       _ , transformed_signal =  scipy.signal.istft(one_hot_transformed_stfts[bomb_num], fs=source0_sample_rate, nperseg = 1024, noverlap = 512)[:len(source0_data)]
    #    output[bomb_num ] = transformed_signal
    #    output[bomb_num] = scipy.signal.savgol_filter(transformed_signal, 5 ,4  )        
       output[bomb_num] = scipy.signal.savgol_filter(transformed_signal, 101, 2)
    last = output.astype(np.int16)
   
    
    
    return last


# sources = duet_source_separation("dataset3", 3)
# Audio(sources[0], rate = 22050)



Time length:  119
Frequency length :  513
stft_0 shape:  (513, 119)
cluster labels:  [0 0 0 ... 0 0 0]
Cluster Centers:  [[ 6.411754e-06]
 [-5.374541e-04]
 [ 5.086695e-04]]


In [34]:
# Audio(sources[1],rate = 22050)

In [35]:
# Audio(sources[2],rate = 22050)

---
## Running and Testing
Use the cell below to run and test your code, and to get an estimate of your grade.

In [6]:
def calculate_score(calculated, expected):
    student_result = set()
    calculated = np.array(calculated)
    if calculated.shape[0] != len(expected):
      return 0, {'Incorrect number of sources!'}
    for i in range(calculated.shape[0]):
        scipy.io.wavfile.write("temp.wav",22050,calculated[i,:])
        r = sr.Recognizer()
        with sr.AudioFile("temp.wav") as source:
            audio = r.record(source)
        try:
            text = r.recognize_sphinx(audio)
            student_result.add(text.lower())
        except:
            student_result.add("Sphinx could not understand audio")
    score = len(student_result.intersection(expected))/len(expected)
    return score, student_result
     
if __name__ == '__main__':
    groundtruth = [{"hello how are you"}, {"nice to meet you","how are you"}, {"how are you","good morning","nice to meet you"}]
    
    output = [['Dataset', 'Expected Output', 'Your Output', 'Grade', 'Points Awarded']]
    for i in range(1,4):
        directory_name = 'dataset{}'.format(i)
        student_output = duet_source_separation(directory_name, i)
        result = calculate_score(student_output, groundtruth[i-1])   
        output.append([
            str(i),
            str(groundtruth[i-1]), 
            str(result[1]), 
            "{:2.2f}%".format(result[0] * 100),
            "{:1.2f} / 5.0".format(result[0] * 5),
        ])

    output.append([
        '<i>👻 Hidden test 1 👻</i>', 
        '<i>???</i>', 
        '<i>???</i>', 
        '<i>???</i>', 
        "<i>???</i> / 10.0"])
    output.append([
        '<i>...</i>', 
        '<i>...</i>', 
        '<i>...</i>', 
        '<i>...</i>', 
        "<i>...</i>"])
    output.append([
        '<i>👻 Hidden test 7 👻</i>', 
        '<i>???</i>', 
        '<i>???</i>', 
        '<i>???</i>', 
        "<i>???</i> / 10.0"])
    display_table(output)

Time length:  80
Frequency length :  513
stft_0 shape:  (513, 80)
cluster labels:  [0 0 0 ... 0 0 0]
Cluster Centers:  [[-1.0222726e-05]
 [ 3.2738238e-04]]
Time length:  119
Frequency length :  513
stft_0 shape:  (513, 119)
cluster labels:  [0 0 0 ... 0 0 0]
Cluster Centers:  [[-2.4270237e-05]
 [ 1.0453977e-04]
 [-3.5316849e-04]]


---
## Rubric
You will be graded on the three data points provided to you (5 points each) and seven additional data points under different settings(10 points each). We will use the same code from the **Running and Testing** section above to grade all 10 traces of data. We will run ASR on your output to see if it generates the corrected separated speech signal. Output order does not matter. Percentage of grade for each data point is based on how many sources you estimated correctly (i.e., assume there are n sources, then you will get $\frac{1}{n} * 100\%$ for each correctedly estimated source).

---
## Submission Guidlines
This Jupyter notebook (`MP2.ipynb`) is the only file you need to submit on Gradescope. As mentioned earlier, you will only be graded using your implementation of the `duet_source_separation` function, which should only return the calculated **NOT** output any plots or data. 

**Make sure any code you added to this notebook, except for import statements, is either in a function or guarded by `__main__`(which won't be run by the autograder). Gradescope will give you immediate feedback using the provided test cases. It is your responsibility to check the output before the deadline to ensure your submission runs with the autograder.**